In [1]:
from __future__ import print_function, unicode_literals
import sys
import json

sys.path.append(".")
import argparse
from tqdm import tqdm
import torch
import subprocess
import numpy as np
import os

from src.models.my_wrap import Wrap_Resnet
from src.models.unsupervised.hybrid2_model import Hybrid2Model
from testing.fh_utils import (
    json_load,
    db_size,
    get_bbox_from_pose,
    read_img,
    convert_order,
    move_palm_to_wrist,
    modify_bbox,
    preprocess,
    create_affine_transform_from_bbox,
)

BBOX_SCALE = 0.33
CROP_SIZE = 224
# DS_PATH = "/hdd/Datasets/freihand_dataset/"
DS_PATH = "/home/dlc/cll/peclr/data/raw/freihand_dataset"

from pprint import pformat
from easydict import EasyDict as edict
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import CometLogger
from src.constants import (
    COMET_KWARGS,
    HYBRID2_CONFIG,
    SIMCLR_CONFIG ,
    BASE_DIR,
    TRAINING_CONFIG_PATH,
)
from src.data_loader.data_set_cbg import Data_Set
from src.data_loader.utils import get_data_cbg, get_train_val_split

from src.experiments.utils import (
    get_callbacks,
    get_general_args,
    get_model,
    prepare_name,
    save_experiment_key,
    update_model_params,
    update_train_params,
)
from src.utils import get_console_logger, read_json

In [8]:
experiment_type = "hybrid2"
# experiment_type = "simclr"
console_logger = get_console_logger(__name__)
args=[ 
                "--color_jitter",
                "--random_crop",
                "--rotate", 
                "--crop",
                "-resnet_size", "50",  
                "-sources", "freihand", 
                "--resize",   
                "-epochs", "500", 
                "-batch_size","128",  
                "-accumulate_grad_batches", "16", 
                "-save_top_k", "1",  
                "-save_period", "1",   
                "-num_workers", "8"
                # // "-d", "./kodim",
                # // "--epochs","300", 
                # // "-lr", "1e-4", 
                # // "--batch-size", "1",
                # // "--cuda", "--save",
                # // "--test-batch-size", "1"
            ]
# get_general_args("Hybrid model 2 training script.")
# args = get_general_args("Simclr model training script.")
train_param = edict(read_json(TRAINING_CONFIG_PATH))
train_param = update_train_params(args, train_param)
# model_param_path = SIMCLR_CONFIG
model_param_path = HYBRID2_CONFIG  # SIMCLR_CONFIG
model_param = edict(read_json(model_param_path))
console_logger.info(f"Train parameters {pformat(train_param)}")
seed_everything(train_param.seed)
data = get_data_cbg(
        Data_Set, train_param, sources=args.sources, experiment_type=experiment_type
    )
model_param = update_model_params(model_param, args, len(data), train_param)
model_param.augmentation = [
key for key, value in train_param.augmentation_flags.items() if value
]
console_logger.info(f"Model parameters {pformat(model_param)}")
model = get_model(
experiment_type="hybrid2",#"simclr"
heatmap_flag=args.heatmap,
denoiser_flag=args.denoiser,
)(config=model_param)
model=Hybrid2Model(config=model_param)

AttributeError: 'list' object has no attribute 'train_ratio'

In [10]:
model_path = '/home/zlc/cll/code/peclr_cbg/data/models/hybrid2-frei-bs256/epoch=286.ckpt'
dev = torch.device("cuda")
checkpoint=torch.load(model_path)
model=Hybrid2Model(config=model_param)
model.load_state_dict(checkpoint['state_dict'])

NameError: name 'model_param' is not defined

In [5]:
checkpoint.keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'callbacks', 'optimizer_states', 'lr_schedulers', 'native_amp_scaling_state', 'state_dict'])

In [6]:
checkpoint['callbacks']

{src.models.callbacks.model_checkpoint.UpdatedModelCheckpoint: {'best_model_score': tensor(4.2962, device='cuda:1'),
  'best_model_path': '/home/zlc/cll/code/peclr_cbg/data/models/hybrid2-frei-bs256/43e469c718b94cbb870c593209820b72/checkpoints/epoch=286.ckpt'}}